In [10]:
import numpy as np
import itertools as it

In [11]:
#pip install import_ipynb

In [12]:
import import_ipynb
import arnoldi

In [13]:
from numpy.linalg import inv

In [14]:
import matplotlib.pyplot as plt

In [15]:
#kronecker-delta function

def k_delta(n,m) :
    if n == m :
        return 1
    else :
        return 0

In [16]:
# define states 
def state(mN, pN) : # num of states of mRNA, num of states of protein

    # states for mRNA and protein
    s = []
    for i in range(mN) :
        for j in range(pN) :
            s.append(np.array([i,j]))
    s = np.array(s)

    return s

In [17]:
# stochastic matrix
def stochastic_matrix(mN, pN , state, avg_m, avg_p, alpha =5, delta = 1) :
    
    # production coefficient of mRNA
    alpha = alpha
    # production coefficient of protein
    betta = avg_p/avg_m
    # degradation coefficient of mRNA
    gamma = alpha/avg_m
    # degradation coefficient of protein
    delta = delta # unit value

    N = mN*pN

    M = np.zeros((N,N))

    Im = np.eye(mN+1)
    Ip = np.eye(pN+1)

    # state
    s = state

    for i in range(N) :
        for j in range(N) :
        
            M[i,j] = alpha*(Im[s[i,0],s[j,0]]*Ip[s[i,1],s[j,1]] - Im[s[i,0]-1,s[j,0]]*Ip[s[i,1],s[j,1]])
            M[i,j] = M[i,j] + betta*s[j,0]*(Im[s[i,0],s[j,0]]*Ip[s[i,1],s[j,1]] - Im[s[i,0],s[j,0]]*Ip[s[i,1]-1,s[j,1]])
            M[i,j] = M[i,j] + gamma*(s[i,0]*Im[s[i,0],s[j,0]]*Ip[s[i,1],s[j,1]] - s[j,0]*Im[s[i,0]+1,s[j,0]]*Ip[s[i,1],s[j,1]])
            M[i,j] = M[i,j] + delta*(s[i,1]*Im[s[i,0],s[j,0]]*Ip[s[i,1],s[j,1]] - s[j,1]*Im[s[i,0],s[j,0]]*Ip[s[i,1]+1,s[j,1]])

    #normalization
    for i in range(N) :
        M[i,i] = 0
        M[i,i] = -np.sum(M[:,i])

    return M
    

In [18]:
def inv_matrix(M) :
    try :
        return inv(M)
    
    except :
        print(f'{M} is singular.')
        return M

In [19]:
# find state at t
def state_t(state_i, t , stochastic_M) :

    N = len(state_i)


    # find smallest eigenvalue 
    inv_M = inv_matrix(stochastic_M)
    e_val, e_vec = arnoldi.QR_algorithm(inv_M,100)
    e_val = 1/e_val #eigenvalue of original matrix

    # transit to eigenvector space coordinate
    state_ei = inv(e_vec).dot(state_i)

    # final state
    state_f = 0
    for i in range(N) :
        state_f += state_ei[i]*e_vec[i]*np.exp(-e_val[i]*t)

    return state_f



In [20]:
# plot for mRNA, protein
def solution(state_f, mN, pN) :
    sol = np.zeros((mN, pN))
    for i in range(mN) :
        for j in range(pN) :
            sol[i,j] = state_f[mN*i+j]/np.sum(np.abs(state_f))

    return sol
    